<a href="https://colab.research.google.com/github/FabioRovai/Personalisation-and-Machine-Learning/blob/main/Assesment3(week7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np

In [2]:

#I implemented an RNN trained on Movies classification tasks to evaluate if these reviews where good or not.
#Also, because the module was working just fine, I stored it in a .py file, keeping this notebook as clean as possible.
!wget 'https://victorzhou.com/movie-reviews-dataset.zip' -qq
!unzip -qq '/content/movie-reviews-dataset.zip' 
!pip install deepctr -qq
#move to main colab directory
%cd '/content/'
#I did a module conversion of this tutorial https://victorzhou.com/blog/keras-rnn-tutorial/ and stored it into a git.
!wget 'https://raw.githubusercontent.com/FabioRovai/test7/main/untitled49.py'
#change into correct directory, otherwse manually change train and test
%cd '/content/movie-reviews-dataset'
#run the module
%run '/content/untitled49.py'

     |████████████████████████████████| 122kB 12.3MB/s 
     |████████████████████████████████| 2.9MB 18.2MB/s 
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
/content
--2021-06-07 14:02:57--  https://raw.githubusercontent.com/FabioRovai/test7/main/untitled49.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2710 (2.6K) [text/plain]
Saving to: ‘untitled49.py’

untitled49.py       100%[===================>]   2.65K  --.-KB/s    in 0s      

2021-06-07 14:02:57 (35.8 MB/s) - ‘untitled49.py’ saved [2710/2710]

/content/movie-reviews-dataset
Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'I imagine that the young people involved in the making of "Necromancy" (aka "The

In [3]:
model.evaluate(test_data)

782/782 [==============================] - 14s 17ms/step - loss: 0.5381 - accuracy: 0.7735


[0.5381135940551758, 0.7734799981117249]

In [4]:
import pandas as pd 
import numpy as np
##Load the data 
#If raw token expires upload manually upload the dataset, is a bug or a security protocol from git.arts
df = pd.read_json("https://git.arts.ac.uk/raw/lmccallum/Personalisation/master/data/renttherunway_cleaned.json?token=AAAAAH7XGLI4G4S6DYUDL5DAYPTSO")
pd.set_option('display.max_colwidth', None)
#df[['review_text']]
#Select random result 
foo=df[['review_text']].sample()
print (foo)
print(model.predict([foo]))
foo=df[['review_text']]
#creating a new section for the review scores
df["review_score"] = model.predict([foo]) 
#print(df["review_score"].describe())
#df["rating"].describe()


                                                                                                                                                                       review_text
16281  This dress is great for us pear-shaped women with big hips, big butt, and heavy thighs.  I loved how this dress made me look smaller and I received a lot of compliments.  
[[0.97447294]]


In [5]:
df.head(2)

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,review_score
0,fit,420272,34d,2260466,137lbs,10.0,vacation,"An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016",0.961109
1,fit,273551,34b,153475,132lbs,10.0,other,"I rented this dress for a photo shoot. The theme was ""Hollywood Glam and Big Beautiful Hats"". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event.",straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013",0.972715


In [6]:
#How many interactions in total, how many unique users, how many unique items 
len(df), len(df["user_id"].unique()), len(df["item_id"].unique())

(192544, 105571, 5850)

### Dates 

Since sequence is important to us, we're going to need to sort the data by review date. Currently, ``Pandas`` sees this column as an ``object``, so we'll use ``pd.to_datetime()`` to convert the string to a date. We can then sort it, and do maths with it! 

In [7]:
#Parse the dates
df["review_date"] = pd.to_datetime(df["review_date"])


In [8]:
#Confirm the data types are correct
df.dtypes

fit                       object
user_id                    int64
bust size                 object
item_id                    int64
weight                    object
rating                   float64
rented for                object
review_text               object
body type                 object
review_summary            object
category                  object
height                    object
size                       int64
age                      float64
review_date       datetime64[ns]
review_score             float32
dtype: object

In [9]:
#Sort
df = df.sort_values(by="review_date")

In [10]:
df_copy=df.copy()

### Getting the User Sequence 

Currently, our dataset has each item on a separate row. In order to get the sequence of purchase for each user, we need to format the data. 

We also take the date in its absolute form and change it to ``days since end of dataset`` (the most recent rental being ``0 days``)

In [11]:
users = df["user_id"].unique()

In [12]:
data = []
last_date = df["review_date"].max()

for user in users:
    ##Get all the items for that user
    rows = df[df["user_id"]==user]
    
    #Get all the item_ids
    items = rows["item_id"]
    #get all the review score
    review_score = rows["review_score"]
    #get all the rating
    rating   = rows["rating"]
 
    
    #Get all the dates
    days_since_now = (last_date - rows["review_date"])
    days_since_now = np.array([i.days for i in days_since_now])
    
    #Collect into a dictionary for each user
    data.append({"item_id":items.values,"nb_days":days_since_now, "rating":rating.values, "reviews":review_score.values })

In [13]:
#Convert to a dataframe and save
data = pd.DataFrame(data)

In [14]:
magic=data.copy()

In [15]:
#adjusting notation
magic["reviews"] = 10 * magic["reviews"]

In [16]:
#data
#magic
data=magic.copy()

In [17]:
magic

,item_id,nb_days,rating,reviews
0,"[125564, 183200]","[2623, 1383]","[10.0, 10.0]","[9.750337, 0.28816873]"
1,"[126335, 132738, 130259]","[2520, 2096, 1745]","[8.0, 10.0, 10.0]","[9.907851, 8.0528345, 8.4637785]"
2,[126335],[2511],[10.0],[9.75249]
3,"[125564, 240137, 468020]","[2510, 848, 848]","[10.0, 10.0, 10.0]","[9.751079, 7.4201574, 9.753986]"
4,[190529],[2500],[10.0],[6.6679835]
...,...,...,...,...
105566,[454564],[3],[8.0],[7.35361]
105567,[1498329],[3],[10.0],[9.747259]
105568,[2835159],[3],[10.0],[9.699287]
105569,[1969604],[3],[10.0],[6.2292433]


### Make the Dataset 

Now we need to make the actual **training set** that we will use for our model. 

Remember, the purpose of this model is to learn to predict the **next item rented in the sequence**. So for our input we will have 

 * [item1, item2, item3, item4, ..., itemN]
 
 * [days1, days2, days3, days4, ..., daysN]
 
 
And for our output we will have **the next item in the sequence** 
  
 
 * [itemN+1]
 
 
We also need our sequences to all be **of equal length**, and because this dataset doesn't have loads of really long sequences, we don't want to throw away stuff below a threshold! So, instead we **zero padd** the end of each sequence if its not as long as the maximum length we have picked (in this case ``5``)

* [0, 0, item1, item2, ..., itemN]
 
* [0, 0, days1, days2, ..., daysN]

In [18]:
from keras.preprocessing.sequence import pad_sequences
#!!!

In [19]:
#split into examples of 5 last things + 1, if less than 5, zero pad
training_set = []
max_len = 5
for _, user in data.iterrows():
    i = 0
    #Get all items for each user 
    items = user["item_id"]
    num_items = len(items)
    #If only one item purchased, there is no sequence! We need at least 2
    #I've found better accuracy limiting to min 3
    if num_items > 2:
        nb_days = np.array(user["nb_days"])
        rating = np.array(user["rating"])
        review_score = np.array(user["reviews"])
        end = False
        #Cycle over items, taking windows of 5 and moving forwards by 1 each time
        while not end:
            
            #If we're off the end of the list, break out of the loop
            target = i + max_len
            if target >= num_items - 1:
                end = True
                target = num_items - 1
            
            #Get the input item and day features, zero padding
            input_items = pad_sequences([items[i:target]], max_len)[0]
            days = pad_sequences([nb_days[i:target]], max_len)[0]
            input_rating = pad_sequences([rating[i:target]], max_len)[0]
            input_rewiews = pad_sequences([review_score[i:target]], max_len)[0]
            
            #Get the adjusted seqeunce (shifted by one) for the target
            target_items = np.concatenate((input_items[1:],[items[target]]))
            #target_items = [items[target]]
            
            #Add to the dataset
            row = {
                "item_id":input_items,
                "nb_days":days,
                "target":target_items,
                "rating":input_rating,
                "reviews":input_rewiews,
            }
            training_set.append(row)
            
            #Increment pointer
            i = i + 1

In [20]:
training_set = pd.DataFrame(training_set)

In [21]:
len(training_set)

34588

In [22]:
training_set

,item_id,nb_days,target,rating,reviews
0,"[0, 0, 0, 126335, 132738]","[0, 0, 0, 2520, 2096]","[0, 0, 126335, 132738, 130259]","[0, 0, 0, 8, 10]","[0, 0, 0, 9, 8]"
1,"[0, 0, 0, 125564, 240137]","[0, 0, 0, 2510, 848]","[0, 0, 125564, 240137, 468020]","[0, 0, 0, 10, 10]","[0, 0, 0, 9, 7]"
2,"[0, 0, 0, 126335, 531077]","[0, 0, 0, 2478, 992]","[0, 0, 126335, 531077, 253667]","[0, 0, 0, 10, 10]","[0, 0, 0, 9, 7]"
3,"[0, 0, 126335, 1338469, 2261828]","[0, 0, 2474, 1105, 1105]","[0, 126335, 1338469, 2261828, 1846462]","[0, 0, 4, 8, 10]","[0, 0, 6, 7, 0]"
4,"[0, 0, 126335, 180014, 833666]","[0, 0, 2458, 1119, 735]","[0, 126335, 180014, 833666, 640839]","[0, 0, 10, 10, 10]","[0, 0, 9, 9, 6]"
...,...,...,...,...,...
34583,"[0, 0, 0, 715164, 2720289]","[0, 0, 0, 4, 4]","[0, 0, 715164, 2720289, 2665815]","[0, 0, 0, 10, 8]","[0, 0, 0, 7, 6]"
34584,"[0, 0, 0, 872442, 322704]","[0, 0, 0, 4, 4]","[0, 0, 872442, 322704, 844580]","[0, 0, 0, 8, 8]","[0, 0, 0, 6, 6]"
34585,"[0, 0, 2893615, 2072280, 2675545]","[0, 0, 4, 4, 4]","[0, 2893615, 2072280, 2675545, 1434889]","[0, 0, 8, 8, 10]","[0, 0, 7, 7, 7]"
34586,"[0, 0, 0, 1252971, 2945301]","[0, 0, 0, 4, 4]","[0, 0, 1252971, 2945301, 1793377]","[0, 0, 0, 10, 8]","[0, 0, 0, 7, 6]"


## Feature Engineering 

### Item Ids

Currently the ``item_ids`` are arbitrary and relate to the **Rent the Runway** catalogue. We are going to encode the in an **embedding** so need to make them indexes from ``0 -> num_items``

In [23]:
#Get all the unique items from the data set
#!!!
all_input_items = np.array([i for i in training_set["item_id"].values]).flatten()
all_target_items = np.array([i for i in training_set["target"].values]).flatten()
all_items = np.concatenate((all_input_items, all_target_items))
unique_items = np.unique(all_items)

In [24]:
#Make a look up dictionary from item_id to index
item_to_index = {item_id:i for i, item_id in enumerate(unique_items)}

In [25]:
#Swap out the new ids and convert to 2d arrays for use in training 
item_indexes = np.array([np.array([item_to_index[i] for i in ids], dtype=int) for ids in training_set["item_id"]], dtype=int)
target_indexes = np.array([np.array([item_to_index[i] for i in ids], dtype=int) for ids in training_set["target"]], dtype=int)
days = np.array([np.array([j for j in i],dtype=int) for i in training_set["nb_days"]],dtype=int)
rating = np.array([np.array([k for k in i],dtype=int) for i in training_set["rating"]],dtype=int)
reviews = np.array([np.array([l for l in i],dtype=int) for i in training_set["reviews"]],dtype=int)

### Bucketting Dates

One interesting approach they have taken at Decathalon is to **bucket** days and then **learn an embedding**, almost treating it as a **categorical variable**. 

To do this, we use ``tf.keras.layers.experimental.preprocessing.Discretization()`` to separate the days into **100 equally spaced bins**, with one more for the **zero padding** 

In [26]:
import tensorflow as tf

In [27]:
#Get min and max
days_min = days.min()
days_max = days.max()

#Generate 100 equally spaced boundaries 
boundaries = list(np.linspace(days_min, days_max,100,dtype=int))
boundaries.insert(0,0)
boundaries[1] = 1

#Bucket the day features
discretize_layer = tf.keras.layers.experimental.preprocessing.Discretization(
    bins=boundaries)
bucket_days = discretize_layer(days).numpy() #- 1
bucket_days = bucket_days

discretize_layer = tf.keras.layers.experimental.preprocessing.Discretization(
    bins=boundaries)
bucket_rat = discretize_layer(rating).numpy() #- 1
bucket_rat = bucket_rat


discretize_layer = tf.keras.layers.experimental.preprocessing.Discretization(
    bins=boundaries)
bucket_rev = discretize_layer(reviews).numpy() #- 1
bucket_rev = bucket_rev

In [28]:
days

array([[   0,    0,    0, 2520, 2096],
       [   0,    0,    0, 2510,  848],
       [   0,    0,    0, 2478,  992],
       ...,
       [   0,    0,    4,    4,    4],
       [   0,    0,    0,    4,    4],
       [   0,    0,    0,    3,    3]])

In [29]:
bucket_days

array([[  0,   0,   0, 100,  84],
       [  0,   0,   0, 100,  35],
       [  0,   0,   0,  99,  40],
       ...,
       [  0,   0,   2,   2,   2],
       [  0,   0,   0,   2,   2],
       [  0,   0,   0,   2,   2]], dtype=int32)

## Training and Validation Sets 

Next we split the data into training and validation sets 

In [30]:
#Generate and shuffle the indexes
total = len(training_set)
all_indexes = np.arange(total)
np.random.shuffle(all_indexes)

#Split the indexes
split = 0.9
train_indexes = all_indexes[:int(total*split)]
test_indexes = all_indexes[int(total*split):]

In [31]:
#Make a dictionary for the inputs 
train_x = {'item_id': item_indexes[train_indexes],
          'nb_days': bucket_days[train_indexes],
           'rating': bucket_rat[train_indexes],
           'reviews':bucket_rev[train_indexes]}

test_x = {'item_id': item_indexes[test_indexes],
          'nb_days': bucket_days[test_indexes],
          'rating': bucket_rat[test_indexes],
          'reviews':bucket_rev[test_indexes]}

#Make an array for the outputs 
train_y = target_indexes[train_indexes]
test_y = target_indexes[test_indexes]

In [32]:
test_y

array([[  63, 2811, 3126, 1680,   13],
       [  34, 3537,  455,  791, 1609],
       [1135, 3393,  614, 2861, 3080],
       ...,
       [2627, 3562, 2274, 2544,  546],
       [   0,    0,  245,  122,  521],
       [ 999,  536, 5052, 2333, 4150]])

In [33]:
test_x

{'item_id': array([[ 125,   63, 2811, 3126, 1680],
        [1634,   34, 3537,  455,  791],
        [4502, 1135, 3393,  614, 2861],
        ...,
        [1128, 2627, 3562, 2274, 2544],
        [   0,    0,    0,  245,  122],
        [3183,  999,  536, 5052, 2333]]),
 'nb_days': array([[55, 55, 46, 23, 23],
        [29, 29,  8,  6,  2],
        [ 8,  5,  5,  5,  4],
        ...,
        [11, 11,  9,  9,  9],
        [ 0,  0,  0, 25, 25],
        [16, 16,  5,  5,  5]], dtype=int32),
 'rating': array([[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        ...,
        [2, 2, 2, 2, 2],
        [0, 0, 0, 2, 2],
        [2, 2, 2, 2, 2]], dtype=int32),
 'reviews': array([[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        ...,
        [2, 2, 2, 2, 0],
        [0, 0, 0, 2, 2],
        [2, 2, 2, 2, 2]], dtype=int32)}

## Build and Train the Model 

Previously we have used ``Keras's Sequential API``, where we first make a model then **sequentially add layers to it** one by one. This works because each layer only has one input, and outputs directly into the next. 

However, whilst this is broadly true of our model, its not strictly true. Our **Embedding layers** both feed into the **Concatenate layer**. So instead, we will use the [Keras Functional API](https://keras.io/guides/functional_api/).

As the documentation says 

```
The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.
```

Essentially, when we build up the ``network graph``, instead of **adding things to the model**, we **specify input layer** we want this new layer to have. 

Here, we make a layer (``x``), and then a layer(``output``), specifying that ``x`` is in the input for ``output``

```
x = layers.Dense(64, activation="relu")
outputs = layers.Dense(10)(x)
```

In [34]:
#Model hyper parameters 
item_vocab_size = len(unique_items)
hp = {
    "embedding_item":100,
    "embedding_rating":20,
    "embedding_reviews":20,
    "embedding_nb_days":20,
    "rnn_units_cat":[1024,512],
    "learning_rate":0.01
}

### ``tf.keras.Input()``

The [Input Layer](https://keras.io/api/layers/core_layers/input/) sits at the base of our ``Model`` and takes does what it says on the tin - Takes the input. 

We specfiy a dictionary of inputs to match our dictionaries we made in the **training set**. This means we have **two separate inputs**, that feed into **two separate embedding layers**.

We give them a ``batch_input_shape`` of ``[None, max_len]``, so that it know each batch will be a sequence of ``max_len`` (in our ase 5) items, but the batch_size itself isn't decided until we ``compile()`` the model. 

In [35]:
inputs = {}
inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                   name='item_id', dtype=tf.int32)

# nb_days bucketized
inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                   name='nb_days', dtype=tf.int32)
#rating
inputs['rating'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                name='rating', dtype=tf.int32)
#reviews   
inputs['reviews'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                   name='reviews', dtype=tf.int32)


### ``tf.keras.layers.Embedding()``

Then we add the embedding layers, each time specifying which item from the **input dictionary** to take as input. 

The **item embedding** takes an input the size of the **number of unique items (vocab size)** and learns a mapping to a denser embedding of a given size. 

The **days embedding** takes an input the size of the **number of buckets + 1 (for zero padding)** and learns a mapping to a denser embedding of a given size. 


In [36]:
embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                           output_dim=hp.get('embedding_item'),
                                           name='embedding_item'
                                          )(inputs['item_id'])
# nbins=100, +1 for zero padding
embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                              output_dim=hp.get('embedding_nb_days'),
                                              name='embedding_nb_days'
                                             )(inputs['nb_days'])


embedding_rating = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                           output_dim=hp.get('embedding_rating'),
                                           name='embedding_rating'
                                          )(inputs['rating'])


embedding_reviews = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                           output_dim=hp.get('embedding_reviews'),
                                           name='embedding_reviews'
                                          )(inputs['reviews'])

In [37]:
print(embedding_item,embedding_nb_days,embedding_rating,embedding_reviews)

KerasTensor(type_spec=TensorSpec(shape=(None, 5, 100), dtype=tf.float32, name=None), name='embedding_item/embedding_lookup/Identity_1:0', description="created by layer 'embedding_item'") KerasTensor(type_spec=TensorSpec(shape=(None, 5, 20), dtype=tf.float32, name=None), name='embedding_nb_days/embedding_lookup/Identity_1:0', description="created by layer 'embedding_nb_days'") KerasTensor(type_spec=TensorSpec(shape=(None, 5, 20), dtype=tf.float32, name=None), name='embedding_rating/embedding_lookup/Identity_1:0', description="created by layer 'embedding_rating'") KerasTensor(type_spec=TensorSpec(shape=(None, 5, 20), dtype=tf.float32, name=None), name='embedding_reviews/embedding_lookup/Identity_1:0', description="created by layer 'embedding_reviews'")


### `` tf.keras.layers.Concatenate()``

We then concatentate embedding layers into one layer 

In [38]:
# Concatenate embedding layers
concat_embedding_input = tf.keras.layers.Concatenate(
 name='concat_embedding_input')([embedding_item, embedding_nb_days,embedding_rating, embedding_reviews  ]) # embedding_rating, embedding_reviews

concat_embedding_input = tf.keras.layers.BatchNormalization(
 name='batchnorm_inputs')(concat_embedding_input)

### LSTM Layers 

When then put in the ``tf.keras.layers.LSTM()`` layer, with a ``tf.keras.layers.BatchNormalization()`` either side. 

More on that in the lecture!

In [39]:
input_layer = concat_embedding_input

for i, num_units in enumerate(hp.get('rnn_units_cat')):
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=num_units,
                                   return_sequences=True,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat' + str(i)
                                   )(input_layer)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm' + str(i))(rnn)
    
    input_layer = rnn

# create encoding padding mask
encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

# Self attention so key=value in inputs
att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                name='attention')(inputs=[rnn, rnn],
                                                  mask=[encoding_padding_mask,
                                                        encoding_padding_mask])



### The Output

Finally, we bring it all together in a ``tf.keras.layers.Dense()`` **softmax layer**. This means that the output of this layer will be 

``
[batch_size x max_len x item_vocab_size]
``

Where each sequence in the batch is a ``[max_len x item_vocab_size]`` tensor telling us the probability of that item in the catalogue being next in the sequence 

In [40]:
# Last layer is a fully connected one
output = tf.keras.layers.Dense(item_vocab_size, activation = tf.nn.softmax, name='output')(att)

### Loss Function 

We write a custom loss function. This is necessary because we need to again mask out the 0s to stop the model optimising towards them. 

In [41]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
def loss_function(real, pred):
    loss = SparseCategoricalCrossentropy()(real, pred)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    return tf.reduce_mean(loss)

### ``tf.keras.Model()``

Finally, we're ready to join the ``Inputs`` and the ``Outputs`` into a ``Model()`` object and ``compile()``.

In [42]:
model = tf.keras.Model(inputs, output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(hp.get('learning_rate')),
    loss=loss_function,
    metrics=['sparse_categorical_accuracy'])

In [43]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
item_id (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
nb_days (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
rating (InputLayer)             [(None, 5)]          0                                            
__________________________________________________________________________________________________
reviews (InputLayer)            [(None, 5)]          0                                            
______________________________________________________________________________________________

In [44]:
history = model.fit(train_x,train_y,
                    epochs=20, 
                    verbose=1,
                    batch_size=512,
                    validation_data=(test_x, test_y))

Epoch 1/20
61/61 [==============================] - 14s 146ms/step - loss: 6.2207 - sparse_categorical_accuracy: 0.0022 - val_loss: 6.3555 - val_sparse_categorical_accuracy: 3.4691e-04
Epoch 2/20
61/61 [==============================] - 8s 127ms/step - loss: 5.5485 - sparse_categorical_accuracy: 0.0111 - val_loss: 6.0281 - val_sparse_categorical_accuracy: 0.0019
Epoch 3/20
61/61 [==============================] - 8s 128ms/step - loss: 4.8224 - sparse_categorical_accuracy: 0.0744 - val_loss: 5.5729 - val_sparse_categorical_accuracy: 0.0101
Epoch 4/20
61/61 [==============================] - 8s 127ms/step - loss: 3.9716 - sparse_categorical_accuracy: 0.2208 - val_loss: 4.8806 - val_sparse_categorical_accuracy: 0.0683
Epoch 5/20
61/61 [==============================] - 8s 126ms/step - loss: 3.2491 - sparse_categorical_accuracy: 0.3724 - val_loss: 4.1614 - val_sparse_categorical_accuracy: 0.2063
Epoch 6/20
61/61 [==============================] - 8s 127ms/step - loss: 2.7342 - sparse_categ

## Predictions 



In [45]:
#Predictions for whole test set
results = model.predict(test_x)

In [46]:
#exmples x max_len x vocab size
results.shape

(3459, 5, 5701)

In [47]:
#max_len x vocab size
results[100].shape

(5, 5701)

In [48]:
#Get index of highest prob in item vocab
results[100][-1].sum()

1.0

In [50]:
results[-1].argmax()

16454

In [51]:
index_to_item = {v:k for k,v in item_to_index.items()}    

In [52]:
index_to_item[3174]

1698166